Reference [kaggle](https://www.kaggle.com/oakmin/audio-data-exploration)

workbook contains plots that may be useful to analyse audio data

In [1]:
%%capture
%pip install numpy==1.21 scipy pandas matplotlib sklearn librosa seaborn speechbrain denoiser

In [4]:
!pip install speechbrain

  Using cached speechbrain-0.5.11-py3-none-any.whl (408 kB)
  Using cached HyperPyYAML-1.0.0-py3-none-any.whl (15 kB)
  Using cached ruamel.yaml-0.17.21-py3-none-any.whl (109 kB)


In [2]:
!which python

'which' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy, matplotlib.pyplot as plt, IPython.display, sklearn
import os
import librosa
import librosa.display
import seaborn as sns
sns.set_style("darkgrid")

import speechbrain
import denoiser

ModuleNotFoundError: No module named 'speechbrain'

In [3]:
DIR="/Users/neelanpather/dev/amr2wav/1/converted/"

In [4]:
from scipy.io.wavfile import read as read_wav
WAV = []
for f in os.listdir(DIR):
    if '.wav' in f:
        sampling_rate, data=read_wav(os.path.join(DIR,f))
        if sampling_rate!=16000:
            print(f"ERROR on {f}")
        print(sampling_rate)
        WAV.append(f)
    else:
        continue
WAV.sort()

16000
16000
16000
16000
16000
16000
16000
16000
16000
16000


In [5]:
WAV

['0000000229520431#-10569#MAGANYAM#LPT-MAGANYAM#20220209100209148.wav',
 '0000000528184405#-10557#PORTIAM13#TCRAMDA6-295#20220209071824490.wav',
 '1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 'amr1.wav',
 'amr2.wav']

In [6]:
k=2
print(WAV[k])
y, sr = librosa.load(os.path.join(DIR,WAV[k]))
IPython.display.Audio(y, rate=sr)

1.wav


# FB research [denoiser](https://github.com/facebookresearch/denoiser/issues/4)

`python3 -m denoiser.enhance --noisy_dir='/Users/neelanpather/dev/amr2wav/1/converted/'/ --out_dir='/Users/neelanpather/dev/amr2wav/1/converted/denoise/'/  --sample_rate 16000`

In [22]:
!python3 -m denoiser.enhance --noisy_dir=DIR --out_dir=DENOISE --sample_rate=16000

INFO:denoiser.pretrained:Loading pre-trained real time H=48 model trained on DNS.


# Speechbrain [MetricGAN-trained model for Enhancement](https://huggingface.co/speechbrain/metricgan-plus-voicebank)

In [25]:
import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank",
)

In [26]:
# Load and add fake batch dimension
noisy = enhance_model.load_audio(
    "/Users/neelanpather/dev/amr2wav/1/converted/3.wav"
).unsqueeze(0)

# Add relative length tensor
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save('/Users/neelanpather/dev/amr2wav/1/converted/enhanced/3.wav', enhanced.cpu(), 16000)

[signal to noise ratio](https://www.garysieling.com/blog/compute-signal-noise-ratio-audio-files-python/)